# Law, Bias, and Algorithms
## Introduction to `R` and `dplyr` (2/2)

In [1]:
# Some initial setup
options(digits = 3, repr.matrix.max.rows = 6)

In this section, and throughout most of the course, we will be using packages that are part of the [`tidyverse`](https://www.tidyverse.org/).

In [2]:
# If you haven't already done so, install tidyverse via
#   install.packages("tidyverse")
# and then load it via:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


The [tidyverse](https://www.tidyverse.org) is a collection of actively-developed `R` packages that follow a certain principle of writing code.
Here, we will primarily focus on `dplyr`, which is the part of `tidyverse` which deals with data manipulation.

First, we will start by reading some data. 
There are many functions in `tidyverse` (and `R` in general) for reading various data formats into the `R` environment as a table of data (or, `data.table`).
For example, either `read_csv` (from `tidyverse`) or `read.csv` (base `R`) can be used to read a `csv` file (e.g., a file where each line represents a row in the table, and the columns are separated by a `,`).

For this course, we have prepared a table of data in the `rds` format.
You can download it (and other data required for this course) by running the `get_data.sh` script, included at the root of this repository.

After downloading the data, we can read the `sqf_sample.rds` file into the current `R` session with the `read_rds` function.
Let's name this table `stops_df`.

In [3]:
# Read the data
stop_df <- read_rds("../data/sqf_sample.rds")

### The data

The loaded data frame, `stop_df`, is a sample of stops in NYC, recorded on a 
[UF-250 form][uf250_link].

Each row corresponds to a stop that an officer made, and the columns represent various entries in the [UF-250 form][uf250_link].


Below is a list of columns in the data, roughly corresponding to the [UF-250 form][uf250_link]:

* Base information regarding stop:
    * `id`, `year`, `date`, `time`, `precinct`, `location_housing`, 
      `suspected_crime`

* Circumstances which led to stop:
    * `stop_reason_object`, `stop_reason_desc`, `stop_reason_casing`,
      `stop_reason_lookout`, `stop_reason_clothing`, `stop_reason_drugs`,
      `stop_reason_furtive`, `stop_reason_violent`, `stop_reason_bulge`,
      `stop_reason_other` 
    
* Suspect demographics:
    * `suspect_dob`, `suspect_id_type`, `suspect_sex`, `suspect_race`,
      `suspect_hispanic`, `suspect_age`, `suspect_height`, `suspect_weight`,
      `suspect_hair`, `suspect_eye`, `suspect_build`, `reason_explained`,
      `others_stopped`

* Whether physical force was used:
    * `force_hands`, `force_wall`, `force_ground`, `force_drawn`,
      `force_pointed`, `force_baton`, `force_handcuffs`,
      `force_pepper`, `force_other`

* Was suspect arrested?: `arrested`

* Was summons issued?: `summons_issued`

* Officer in uniform?: `officer_uniform`, `officer_verbal`, `officer_shield`

* Was person frisked?: `frisked`
    * if yes: `frisk_reason_suspected_crime`, `frisk_reason_weapons`, 
      `frisk_reason_attire`, `frisk_reason_actual_crime`, 
      `frisk_reason_noncompliance`, `frisk_reason_threats`,
      `frisk_reason_prior`, `frisk_reason_furtive`, `frisk_reason_bulge`

* Was person searched?: `searched`,
    * if yes: `searched_hardobject`, `searched_outline`,
      `searched_admission`, `searched_other`

* Was weapon found?: `found_weapon`
    * if yes: `found_gun`, `found_pistol`, `found_rifle`, `found_assault`,
      `found_knife`, `found_machinegun`, `found_other`
      
* Was other contraband found?: `found_contraband`

* Additional circumstances/factors
    * `additional_report`, `additional_investigation`, `additional_proximity`, 
      `additional_evasive`, `additional_associating`, `additional_direction`, 
      `additional_highcrime`, `additional_time`, `additional_sights`, 
      `additional_other`

* Additional reports prepared: `extra_reports`

[uf250_link]: https://www.prisonlegalnews.org/media/publications/Blank%20UF-250%20Form%20-%20Stop%2C%20Question%20and%20Frisk%20Report%20Worksheet%2C%20NYPD%2C%202016.pdf
      

We can also use some base `R` functions to explore the data.

### Exercise: understanding the data

Run `str(stop_df)` and `summary(stop_df)`.
Compare the output to the **UF-250 form** and the column descriptions above.
The goal is to understand what each column means, and how that information is stored (e.g., numbers, strings, logical values)

In [4]:
# WRITE CODE HERE
# START solution
str(stop_df)
summary(stop_df)
# END solution

'data.frame':	100000 obs. of  81 variables:
 $ id                          : int  2686798 2075967 1268296 2522700 2722653 2333983 1680748 3352766 2408866 2187979 ...
 $ year                        : int  2010 2009 2008 2010 2011 2010 2009 2011 2010 2010 ...
 $ date                        : chr  "2010-12-17" "2009-12-08" "2008-07-12" "2010-09-08" ...
 $ time                        : chr  "15:15" "15:19" "14:45" "21:55" ...
 $ precinct                    : Factor w/ 77 levels "1","5","6","7",..: 53 23 7 26 74 37 36 56 49 1 ...
 $ location_housing            : Factor w/ 3 levels "housing","neither",..: 2 2 3 1 2 2 2 2 3 2 ...
 $ suspected_crime             : Factor w/ 11 levels "burglary","cpw",..: 1 2 7 2 10 8 5 2 10 11 ...
 $ stop_reason_object          : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ stop_reason_desc            : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ stop_reason_casing          : logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ stop_reason_lookout         : log

       id               year          date               time          
 Min.   : 978652   Min.   :2008   Length:100000      Length:100000     
 1st Qu.:1492175   1st Qu.:2008   Class :character   Class :character  
 Median :1984018   Median :2009   Mode  :character   Mode  :character  
 Mean   :2033863   Mean   :2009                                        
 3rd Qu.:2490719   3rd Qu.:2010                                        
 Max.   :3386938   Max.   :2011                                        
                                                                       
    precinct     location_housing           suspected_crime  stop_reason_object
 75     : 4089   housing: 7161    cpw               :29087   Mode :logical     
 120    : 3855   neither:78749    robbery           :21230   FALSE:97020       
 14     : 3558   transit:14090    burglary          :10424   TRUE :2980        
 60     : 2920                    grand larceny auto:10005                     
 61     : 2100          

## Introduction to `dplyr` verbs

In the world of `dplyr`, a _verb_ is a function that

* takes a data frame as its first argument, and
* returns another data frame as a result

Any function that meets this criteria, even if it's not necessarily a function in the `dplyr` package, can be considered a _verb_.

For example, the `head()` function in `R`, if applied to a data frame, will return a first `n` rows of a data frame.

In [5]:
head(stop_df, n = 3)  # Return a data frame that consists of the first three rows of stop_df

,id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
322910,2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12
218707,2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12
53361,1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7


In this sense, the base `R` function `head()` is a verb.

The **core idea** of `dplyr` is that a vast majority of data manipulation needs can be satisfied through a combination of five verbs.

verb                 | action
-------------------- | ---------
`filter(df, ...)`    | select a subset of _rows_ by some specified condition
`select(df, ...)`    | select a subset of _columns_
`mutate(df, ...)`    | create a _new column_ (often as a function of existing columns)
`arrange(df, ...)`   | reorder (sort) _rows_ according to values of specific _columns_
`summarize(df, ...)` | aggregate and reduce a vector (column) to a single value

We will explore each of these verbs (and some additional variations within each category) below.

### Selecting rows (1/5)

The `filter(df, ...)` verb is used to select a subset of _rows_ that satisfy the conditions specified in `...`. 
The conditions must be written in a form that would evaluate to either `TRUE` or `FALSE`.

For example, if we want a data frame of stops in precinct `22`,

In [6]:
filter(stop_df, precinct == 22)

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
1392818,2008,2008-10-09,18:20,22,neither,criminal possession of marihuana,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,18,10
1894160,2009,2009-08-11,02:00,22,neither,robbery,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,2,8
2074901,2009,2009-12-07,16:40,22,neither,criminal possession of marihuana,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,16,12
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2267825,2010,2010-04-10,23:00,22,neither,other,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,23,4
2271913,2010,2010-04-13,16:55,22,neither,criminal possession of marihuana,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,16,4
1658966,2009,2009-03-13,16:05,22,neither,robbery,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,16,3


Multiple conditions can also be specified.

For example, if we want a data frame of stops in precinct `22` in which a weapon was found,

In [7]:
filter(stop_df, precinct == 22, found_weapon)
# Note that we could also write
#   filter(stop_df, precinct == 22, found_weapon == TRUE)
# but given that the found_weapon column is already a logical value,
# the "== TRUE" would be redundant.

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
2392428,2010,2010-06-13,15:05,22,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,6


By default, `filter()` will combine multiple conditions as `AND` operations.
In the example above, we are returned stops where `precinct == 22` _**AND**_ `found_weapon == TRUE`.

We can specify an OR condition by using the `|` operator.

For example, if we want stops in precinct `22` where either a weapon _**OR**_ contraband was found,

In [8]:
filter(stop_df, 
       precinct == 22, 
       found_weapon | found_contraband)

# Note that, within a set of parentheses, you can have as many line breaks as you want.
# It's generally easier to read (and modify) if you have each condition on a new line.
# For example, if you quickly wanted to check what the result looks like for _all_ precincts,
# in the above code, you could just "comment out" the precinct == 22 condition by adding a "#" at the begining of the line

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
1232958,2008,2008-06-11,10:38,22,neither,criminal possesion of controlled substance,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,10,6
1081888,2008,2008-03-04,19:20,22,neither,criminal possession of marihuana,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,19,3
1231180,2008,2008-06-09,15:38,22,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,15,6
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1110803,2008,2008-03-21,15:50,22,neither,criminal possession of marihuana,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,15,3
1214689,2008,2008-05-28,13:10,22,neither,criminal possession of marihuana,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,5
1312725,2008,2008-08-16,15:40,22,neither,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,15,8


Use the `%in%` operator to filter to values that match a collection of values. 

For example, suppose we are primarily interested in `burglary`, `robbery`, and `grand larceny`.

In [9]:
filter(stop_df, 
       suspected_crime %in% c("burglary", "robbery", "grand larceny"))

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12
1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7
2722653,2011,2011-01-13,23:28,120,neither,robbery,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,23,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1953331,2009,2009-09-18,11:20,72,neither,robbery,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,11,9
2455369,2010,2010-07-24,20:10,6,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,7
2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7


As a side note, a preferred way of writing the above filter would be to first create a vector of the crimes that we are interested in.
That way, it's easier for us to (1) understand the code and (2) change the code later, e.g., if we decide we're interested in different type of crime.

In [10]:
interesting_crimes <- c("burglary", "robbery", "grand larceny")

filter(stop_df, 
       suspected_crime %in% interesting_crimes)

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12
1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7
2722653,2011,2011-01-13,23:28,120,neither,robbery,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,23,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1953331,2009,2009-09-18,11:20,72,neither,robbery,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,11,9
2455369,2010,2010-07-24,20:10,6,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,7
2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7


Finally, use `!` to negate any condition. 

For example, if we wanted to find all stops where the suspected crime is _not_ one of the `interesting_crimes`,

In [11]:
filter(stop_df, 
       !(suspected_crime %in% interesting_crimes))

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12
2522700,2010,2010-09-08,21:55,43,housing,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,21,9
2333983,2010,2010-05-12,19:55,62,neither,grand larceny auto,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,19,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1891165,2009,2009-08-08,19:56,50,neither,grand larceny auto,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,19,8
2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2
1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2


or, if you want to exclude stops in 2008

In [12]:
filter(stop_df, year != 2008)

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12
2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12
2522700,2010,2010-09-08,21:55,43,housing,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,21,9
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2
1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2
2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7


### Exercise: `filter()`

1. Find all the minority (non-white) race (`suspect_race`) stops which took place in transit (`location_housing == "transit"`), 
   where the suspected crime was criminal possession of a weapon (`cpw`) and a gun was eventually found (`found_gun`).
1. Find all stops that occurred in either 2008 _or_ 2011, in precinct 13, between the hours 0 and 2, 
   where a weapon was found, but no contraband was found

In [13]:
# WRITE CODE HERE
# START solution
# 1. 
filter(stop_df, 
       suspect_race != "white",
       location_housing == "transit",
       suspected_crime == "cpw",
       found_gun)

# 2. 
filter(stop_df, 
       year %in% c(2008, 2011),
       precinct == 13,
       hour >= 0,
       hour <= 2,
       found_weapon,
       !found_contraband)
# END solution

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
1845792,2009,2009-07-08,18:55,14,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,18,7
1421012,2008,2008-10-25,11:54,7,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,11,10
1416833,2008,2008-10-23,15:20,73,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,10
2702721,2011,2011-01-02,01:40,40,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,1,1


id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
3068765,2011,2011-07-09,00:10,13,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7
1021848,2008,2008-01-27,01:50,13,neither,cpw,TRUE,FALSE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,1,1
3068829,2011,2011-07-10,02:30,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,2,7
1420365,2008,2008-10-25,00:45,13,neither,cpw,FALSE,TRUE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,0,10


### Selecting columns (2/5)

Use `select(df, ...)` to either specify which columns to select,

In [14]:
select(stop_df, year, precinct, suspected_crime)

,year,precinct,suspected_crime
322910,2010,83,burglary
218707,2009,40,cpw
53361,2008,13,grand larceny
⋮,⋮,⋮,⋮
99998,2011,13,cpw
99999,2009,60,criminal sale of controlled substance
100000,2010,50,grand larceny


or to specify which columns to exclude, using `-`.

In [15]:
select(stop_df, -id, -year, -date, -time, -precinct, -location_housing)

,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,stop_reason_lookout,stop_reason_clothing,stop_reason_drugs,stop_reason_furtive,stop_reason_violent,stop_reason_bulge,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
322910,burglary,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12
218707,cpw,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12
53361,grand larceny,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
99998,cpw,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2
99999,criminal sale of controlled substance,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2
100000,grand larceny,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7


`dplyr` also provides some useful helper functions to `select()` columns that match specific criteria.

* `starts_with(x)`: match column names that start with `x`
* `ends_with(x)`: match column names that end with `x`
* `contains(x)`: match column names that contain `x`
* `matches(x)`: match column names that match (the regular expression) `x`

where `x` is a string (in either single- or double-quotes).

For example, if we want all the columns indicating what was found,

In [16]:
select(stop_df, starts_with("found_"))

,found_weapon,found_pistol,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband
322910,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
218707,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
53361,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
99998,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE
99999,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


You can see the documentation for `select()` for details.
In general, for any `R` function, you can pull-up the documentation (if one exists) by running `?` followed by the function name.
For example, to see the documentation for `select()` as provided in the `dplyr` package, run:

In [17]:
?dplyr::select

select {dplyr},R Documentation
.data,"A tbl. All main verbs are S3 generics and provide methods for tbl_df(), dtplyr::tbl_dt() and dbplyr::tbl_dbi()."
...,"One or more unquoted expressions separated by commas. You can treat variable names like they are positions. Positive values select variables; negative values to drop variables. If the first expression is negative, select() will automatically start with all variables. Use named arguments to rename selected variables. These arguments are automatically quoted and evaluated in a context where column names represent column positions. They support unquoting and splicing. See vignette(""programming"") for an introduction to these concepts."


### Exercise: `select()`

1. Base on the UF-250 form, select the columns from `stop_df` that the officer could have reasonably filled-out _before_ making the stop

In [18]:
# WRITE CODE HERE
# START solution
# 1. 
select(stop_df, 
       # Base information regarding stop:
       id, year, date, time, precinct, location_housing, suspected_crime,
       # Columns specifying stop reason:
       starts_with("stop_reason_"),
       # Suspect demographics:
       starts_with("suspect_"),
       # Officer information:
       starts_with("officer_"))
# END solution

,id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,suspect_hispanic,suspect_age,suspect_height,suspect_weight,suspect_hair,suspect_eye,suspect_build,officer_uniform,officer_verbal,officer_shield
322910,2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,TRUE,12,5.00,130,brown,brown,medium,FALSE,TRUE,TRUE
218707,2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,19,6.00,170,black,brown,thin,FALSE,TRUE,TRUE
53361,1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,47,5.83,200,brown,brown,medium,FALSE,TRUE,TRUE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
99998,2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,32,5.58,180,black,brown,thin,FALSE,TRUE,TRUE
99999,1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,TRUE,33,6.17,240,brown,brown,medium,FALSE,TRUE,TRUE
100000,2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,22,5.92,180,black,brown,thin,FALSE,TRUE,TRUE


### Create new columns (3/5)

Use `mutate(df, ...)` to create new columns, usually as a function of existing columns.

Suppose we wish to create a column called `is_cpw`, indicating whether the `suspected_crime` for each stop was `cpw` or not.
We would write,

In [19]:
mutate(stop_df, is_cpw = (suspected_crime == "cpw"))
# Note that the new column will be the very last column in the data frame

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month,is_cpw
2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12,FALSE
2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12,TRUE
1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2,TRUE
1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2,FALSE
2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7,FALSE


Within a single `mutate()` function, you can refer to the new columns you've created in the previous argument.

One, probably useless, yet illustrative example would be computing BMI for each pedestrian who was stopped.

Recall that

$$ \mathrm{BMI} = \frac{\mathrm{weight~(kg)}}{\mathrm{height~(m)}^2}, $$

where weight is in kilograms and height is in meters. 

However, in our data, it seems that weight is in pounds and height is in feet.
Hence we must

1. convert weight in pounds to weight in kilogram (by multiplying 0.45)
1. convert height in feet to height in meters (by multiplying 0.31)
1. find the square of height in meters
1. compute weight in kilogram divided by square of height in meters

In [20]:
mutate(stop_df,
       weight_kg = suspect_weight * .45,
       height_m = suspect_height * .31,
       height_sq = height_m^2,
       BMI = weight_kg / height_sq)

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_other,found_gun,found_contraband,extra_reports,hour,month,weight_kg,height_m,height_sq,BMI
2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,15,12,58.5,1.55,2.40,24.3
2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,TRUE,FALSE,FALSE,15,12,76.5,1.86,3.46,22.1
1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,14,7,90.0,1.81,3.27,27.5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,19,2,81,1.73,3.00,27.0
1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,20,2,108,1.91,3.65,29.6
2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,0,7,81,1.83,3.36,24.1


### Exercise: `mutate()`

1. Create a column, named `am_pm`, indicating whether the stop occurred in the morning (prior to 12) or not.
    * This can be created either as a logical column (`TRUE`/`FALSE`), or
    * by using `if_else` to create a character column of `"am"`/`"pm"`

In [21]:
# WRITE CODE HERE
# START solution
# 1. a. as a logical column
mutate(stop_df, am_pm = hour < 12)

# 1. b. as a character column
mutate(stop_df, am_pm = if_else(hour < 12, "am", "pm"))
# END solution

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month,am_pm
2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12,FALSE
2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12,FALSE
1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2,FALSE
1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2,FALSE
2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7,TRUE


id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month,am_pm
2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12,pm
2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12,pm
1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7,pm
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2,pm
1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2,pm
2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7,am


### Sorting (4/5)

Use `arrange(df, ...)` to reorder the rows of a data frame by the value of specified columns.
Multiple conditions are arranged from left to right.

In [22]:
arrange(stop_df, date, time)

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
978699,2008,2008-01-01,00:01,62,neither,criminal trespass,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,1
978703,2008,2008-01-01,00:01,62,neither,burglary,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,1
978701,2008,2008-01-01,00:01,62,neither,burglary,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
3386837,2011,2011-12-31,22:15,25,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,22,12
3386846,2011,2011-12-31,22:20,90,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,22,12
3386938,2011,2011-12-31,22:50,81,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,22,12


Use `desc()` around columns that you want to sort in `desc`ending order.

In [23]:
arrange(stop_df, date, desc(time))

id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
979583,2008,2008-01-01,23:30,120,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,23,1
979558,2008,2008-01-01,23:10,79,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,23,1
979541,2008,2008-01-01,23:00,79,neither,other,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,23,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
3385779,2011,2011-12-31,00:45,122,neither,grand larceny auto,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,12
3385769,2011,2011-12-31,00:40,115,neither,grand larceny auto,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,12
3385741,2011,2011-12-31,00:15,49,neither,other,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,12


### Aggregating (5/5)

Use `summarize(df, ...)` to aggregate multiple rows into a single row. Unlike `mutate()`, function that are used in `summarize()` must return a single value (i.e., "aggregate" the provided vector)

For example, to find the min, mean, and max height of all suspects,

In [24]:
summarize(stop_df,
          min_height = min(suspect_height),
          avg_height = mean(suspect_height),
          max_height = max(suspect_height))

min_height,avg_height,max_height
3,5.74,7.92


`dplyr` also provides a special function `n()` which will evaluate to the number of rows within a `dplyr` verb.

For example, to count how many stops (rows) there are in total, 

In [25]:
summarize(stop_df, N = n())

N
100000


### Exercise: `summarize()`

For `stop_df`, find 

* Average `suspect_weight`,
* Median `suspect_weight`,
* Standard deviation of `suspect_weight`, and
* Number of all stops

In [26]:
# WRITE CODE HERE
# START solution
summarize(stop_df,
          avg_weight = mean(suspect_weight),
          med_weight = median(suspect_weight),
          sd_weight = sd(suspect_weight),
          N = n())
# END solution

avg_weight,med_weight,sd_weight,N
171,170,29.7,100000


### Grouping (Split-apply-combine)

Now that we've covered the five core verbs, we should be able to manipulate data to our heart's desires, yes?

Then, how about:

* The number of stops for each `suspect_race`?
* Proportion of stops in each precinct that resulted in recovery of a weapon?
* Number of stops for each race by `suspected_crime` type?

As an example, let's just consider the number of stops for each `suspect_race`.

A natural, but _**tedious**_ way to compute this would look something like this:

In [27]:
# NOTE: Code in this cell is intended to be an example of a BAD way to compute this.
#       This is purely for illustrative purposes, and should NEVER EVER be re-used, in any context.
white_stops <- filter(stop_df, suspect_race == "white")
black_stops <- filter(stop_df, suspect_race == "black")
hispanic_stops <- filter(stop_df, suspect_race == "hispanic")

N_whites <- summarize(white_stops, N = n())
N_blacks <- summarize(black_stops, N = n())
N_hispanics <- summarize(hispanic_stops, N = n())

c(white = N_whites$N, black = N_blacks$N, hispanic = N_hispanics$N)

white    black hispanic 
   12682    54806    32512

This style of code can easily get out of hand, and would be a nightmare to maintain! 
(e.g., what happens if we get a new dataset that includes 8 different race categories?)

As horrible as the above code is, it is useful in highlighting a common pattern that emerges when manipulating data:

1. **Split**: The data are split into smaller pieces of data, according to one (or more) column. 
   In this case, we've split the data by the `suspect_race` column.
1. **Apply**: Some operation is applied to each of the smaller pieces.
   In this case, we've simply counted the number of rows of each piece using `summarize()` and `n()`.
1. **Combine**: The results of the previous **apply** are combined to some final data structure.
   In the above case, for simplicity, we've combinded the result as a vector; but in practice we usually want to keep everything in the form of a data frame.

This pattern in data manipulation is so common, that there is a `dplyr` verb for it. 
This is the `group_by` verb.

On it's own, `group_by` makes no visible changes to a data frame, other than marking the data frame as being "grouped".
The difference is only made apparent when we apply some other verb to a grouped data frame.

Note that none of the `dplyr` verbs make any changes to the original data frame! This is very intentional. 
So, for now, we need to save the "grouped" data as a new variable for our changes to have effect (but we'll see a more convenient approach to this later).

In [28]:
stops_by_race <- group_by(stop_df, suspect_race)

# Note that the two data frames, on the surface, seem identical.
stop_df
stops_by_race

,id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
322910,2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12
218707,2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12
53361,1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
99998,2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2
99999,1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2
100000,2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7


,id,year,date,time,precinct,location_housing,suspected_crime,stop_reason_object,stop_reason_desc,stop_reason_casing,⋯,found_rifle,found_assault,found_knife,found_machinegun,found_other,found_gun,found_contraband,extra_reports,hour,month
1,2686798,2010,2010-12-17,15:15,83,neither,burglary,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,15,12
2,2075967,2009,2009-12-08,15:19,40,neither,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,15,12
3,1268296,2008,2008-07-12,14:45,13,transit,grand larceny,FALSE,TRUE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,14,7
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
99998,2801672,2011,2011-02-19,19:01,13,transit,cpw,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,19,2
99999,1595066,2009,2009-02-09,20:20,60,neither,criminal sale of controlled substance,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,20,2
100000,2419926,2010,2010-07-01,00:10,50,transit,grand larceny,FALSE,FALSE,TRUE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,0,7


In [29]:
# But we can see a difference when applying, for example, a summarize
summarize(stop_df, N = n())

N
100000


In [30]:
summarize(stops_by_race, N = n())

suspect_race,N
white,12682
black,54806
hispanic,32512


As shown in the simple example above, when a `dplyr` verb is applied to a "grouped" data frame,
`dplyr` internally **splits**-**applies**-and **combines** the data, finally returning results for
_each of the unique values that are found in the columns by which the data frame is grouped_.

This can be a lot to process, if it's the first time you've seen this. 
But once you get used to it (via trying a bunch of manipulation tasks and seeing some more examples), you'll find it extremely convenient and powerful.

Let's try answering the other questions we started this section with.

* Proportion of stops in each precinct that resulted in recovery of a weapon?


In [31]:
stops_by_precinct <- group_by(stop_df, precinct)

summarize(stops_by_precinct, 
          # Note that below, we compute the mean of a logical (TRUE/FALSE) column
          #   this is a common "trick" used in R to compute proportions, taking advantage of
          #   the fact that a TRUE value is equivalent to a 1 and FALSE is equivalent to a 0 in R
          prop_weapons_found = mean(found_weapon))

precinct,prop_weapons_found
1,0.0089
5,0.0229
6,0.0126
⋮,⋮
120,0.01582
122,0.01205
123,0.00794


* Number of stops for each race by `suspected_crime` type?

To find this, we first notice that the data must be split by two columns, `suspect_race` _and_ `suspected_crime`.
This is also supported by `group_by`. In fact, you can have as many grouping variables as you'd like, as long as it makes sense in the context.

In [32]:
stops_by_race_and_crime <- group_by(stop_df, suspect_race, suspected_crime)
summarize(stops_by_race_and_crime, N = n())

suspect_race,suspected_crime,N
white,burglary,2793
white,cpw,1482
white,criminal possesion of controlled substance,585
⋮,⋮,⋮
hispanic,petit larceny,881
hispanic,robbery,7331
hispanic,other,3115


### Exercise: `group_by()`

For `stop_df`, calculate the proportion of people that are arrested (`arrested`) and frisked (`frisked`) for each race group.

In [33]:
# WRITE CODE HERE
# START solution
race_group <- group_by(stop_df, suspect_race)
summarize(race_group, 
          arrest_rate = mean(arrested),
          frisk_rate = mean(frisked)
         )
# END solution

suspect_race,arrest_rate,frisk_rate
white,0.1012,0.576
black,0.0972,0.740
hispanic,0.1013,0.719


### Multiple (chained) operations

As we've breifly seen above, we would often like to apply multiple operations (verbs) to a data frame.
However, by design, verbs do not save intermediate changes to the original data frame, so for each operation we would have to assign the result to a new data frame.

Even for a reasonable number of operations, this can get quite messy (i.e., we'd end up with so many names and data frames that we only use as intermediate steps).

Consider the following query:

* For each suspected crime type, what is the proportion of stops for each race group?

We can think of finding the answer in multiple steps:

1. group by `suspected_crime` and `suspect_race`
1. find the number of stops for each of the groups in the previously grouped data frame
1. with the computed stops for each crime type/race pair, re-group by only `suspected_crime`
1. create a new column which computes the proportion of counts for each row, over the sum of rows

Using the current method of saving all intermediate results, the implementation would look something like this:

In [34]:
# NOTE: Code in this cell is intended to be an example of a BAD implementation.
#       While this implementation is acceptable, it is intended for illustrative purposes, 
#       and is best avoided.
stops_by_crime_and_race <- group_by(stop_df, suspected_crime, suspect_race)
counts_by_crime_and_race <- summarize(stops_by_crime_and_race, N = n())
regroup_by_crime <- group_by(counts_by_crime_and_race, suspected_crime)
mutate(regroup_by_crime, prop = N / sum(N))

suspected_crime,suspect_race,N,prop
burglary,white,2793,0.268
burglary,black,3584,0.344
burglary,hispanic,4047,0.388
⋮,⋮,⋮,⋮
other,white,1402,0.171
other,black,3675,0.449
other,hispanic,3115,0.380


The above code is bad for a multiple reasons. Among others, it's

* creating a lot of unnecessary intermediate results that will not be used again
* difficult to read, if you don't already know what the end goal is (e.g., you eyes have to wander left-to-right-to-left a few times to see what's going on)

A sophisticated, yet quite simple, solution to this problem is the introduction of `%>%`, also called the "pipe operator".

`%>%` is a _binary operator_ (much like `+` or `-`) which, in words, takes the result of the left-hand side, and uses it as the first argument on the right hand side. 
This may be confusing at first, but might make more sense in the context of `dplyr` _verbs_. Recall, a _verb_ in `dplyr` is any function that _returns a data frame_ (LHS) and _takes a data frame as its first argument_ (RHS).

If further notation is helpful, one could also write the `%>%` as
```
f(x) %>% g(y) = g(f(x), y)
```
which, depending on your background, could be clarifying or more confusing.

What this means from a practical standpoint, however, is that we no longer need to _save_ intermediate results just to use them in the next verb. 
Instead, we can use `%>%` to send results from a verb down a "pipe" to the next verb.
Consider our previous example, which involved four verbs, with three intermediate steps.
Using `%>%`, the same result can now be achieved in a (conceptually) single line:

In [35]:
stop_df %>%
    group_by(suspected_crime, suspect_race) %>%
    summarize(N = n()) %>%
    group_by(suspected_crime) %>%
    mutate(prop = N / sum(N))

suspected_crime,suspect_race,N,prop
burglary,white,2793,0.268
burglary,black,3584,0.344
burglary,hispanic,4047,0.388
⋮,⋮,⋮,⋮
other,white,1402,0.171
other,black,3675,0.449
other,hispanic,3115,0.380


Note the intentional style of (1) starting from the data frame (instead of a verb that explicitly includes the data frame) and (2) keeping each verb on its own line. 
This not only makes it easier to read, but also easier to maintain and modify.

### Exercise: Introduction to `dplyr`

1. Find the distribution of suspected crimes for each race group
(e.g., what proportion of white/black/Hispanic pedestrian stops were due to suspicion of `cwp`?)

2. Using the result from the above exercise, find the suspected crimes that account for more than 10% of stops in all race groups.

hint: `all(logical_vector)` will return `TRUE` if all values in `logical_vector` are `TRUE`

In [36]:
# WRITE CODE HERE
# START solution
# 1. 
suspected_crime_by_race <- stop_df %>%
    group_by(suspected_crime, suspect_race) %>%
    summarize(N = n()) %>%
    group_by(suspect_race) %>%
    mutate(proportion = N / sum(N))
suspected_crime_by_race

# 2. 
suspected_crime_by_race %>%
    group_by(suspected_crime) %>%
    filter(all(proportion > 0.1))
# END 

suspected_crime,suspect_race,N,proportion
burglary,white,2793,0.2202
burglary,black,3584,0.0654
burglary,hispanic,4047,0.1245
⋮,⋮,⋮,⋮
other,white,1402,0.1106
other,black,3675,0.0671
other,hispanic,3115,0.0958


suspected_crime,suspect_race,N,proportion
cpw,white,1482,0.117
cpw,black,19404,0.354
cpw,hispanic,8201,0.252
robbery,white,1582,0.125
robbery,black,12317,0.225
robbery,hispanic,7331,0.225


## Ending notes

There are many, MANY more verbs that we simply did not have the time to cover here, but are immensely useful. 
Some examples are:

* `rename(df, ...)`: rename columns
* `slice(df, ...)`: select rows of a data frame by index, instead of some condition
* `top_n(df, N, col)`: retrieve the top N rows for values in some specified column

You are highly recommended to explore more. One great resource for learning about `tidyverse` and using it to work with data is Hadley Wickham's online book: [R for Data Science](https://r4ds.had.co.nz/). 

Hadley Wickham is also the original author for many of the packages in `tidyverse`. In fact, in the "early days" (circa 2016), before the word `tidyverse` was created, the collection of Hadley Wickham's `R` packages were unofficially referred to as the `hadleyverse`, until [Hadley announced tidyverse and explicitly asked people to stop calling it the hadleyverse](https://twitter.com/hadleywickham/status/774008060549312512?lang=en).